In [1]:
import numpy as np
import altair as alt
import pandas as pd
import scipy
from scipy.stats import norm
import math
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
experiments = 10

In [3]:
alpha = 0.05

In [4]:
n = 50

In [5]:
B = 100_000

In [6]:
y_list = np.random.normal(size=n)

In [7]:
x_list = [math.exp(y) for y in y_list]

In [8]:
theta_hat = scipy.stats.skew(x_list)

In [22]:
bootstraps = []
for i in range(B):
    theta_hat_b = scipy.stats.skew(np.random.choice(x_list, replace=True, size=len(x_list)))
    bootstraps.append(theta_hat_b)

In [23]:
se = math.sqrt(np.var(bootstraps))

In [24]:
print("Estimated Standard Error:", se)

Estimated Standard Error: 0.38136019362503243


In [25]:
z_alpha_2 = norm.ppf(1-(alpha/2))

In [26]:
bootstrap_lower_quantile= np.quantile(bootstraps, alpha/2)

bootstrap_upper_quantile = np.quantile(bootstraps, 1-(alpha/2))

In [27]:
normal_confidence_interval = (theta_hat - z_alpha_2 * se, theta_hat + z_alpha_2 * se)
pivotal_confidence_interval = (2*theta_hat - bootstrap_upper_quantile, 2*theta_hat - bootstrap_lower_quantile)
percentile_confidence_interval = (bootstrap_lower_quantile, bootstrap_upper_quantile)

In [28]:
true_skewness = (math.exp(1) + 2)*math.sqrt(math.exp(1) - 1)

In [31]:
print("True Theta:", true_skewness)

True Theta: 6.184877138632554


In [32]:
print("Estimated Theta:", theta_hat)

print(f"{(1-alpha)*100}% Confidence Intervals:")

print("Normal:", normal_confidence_interval)
print("Pivotal:", pivotal_confidence_interval)
print("Percentile:", percentile_confidence_interval)

Estimated Theta: 1.8306584605870668
95.0% Confidence Intervals:
Normal: (1.0832062159447817, 2.578110705229352)
Pivotal: (1.0469209847184486, 2.54437819165088)
Percentile: (1.1169387295232533, 2.614395936455685)


In [34]:
normal_ci_count = 0
pivotal_ci_count = 0
percentile_ci_count = 0

for i in range(experiments):
    y_list = np.random.normal(size=n)
    x_list = [math.exp(y) for y in y_list]
    theta_hat = scipy.stats.skew(x_list)

    true_skewness = (math.exp(1) + 2)*math.sqrt(math.exp(1) - 1)

    bootstraps = []
    for i in range(B):
        theta_hat_b = scipy.stats.skew(np.random.choice(x_list, replace=True, size=len(x_list)))
        bootstraps.append(theta_hat_b)

    se = math.sqrt(np.var(bootstraps))
    z_alpha_2 = norm.ppf(1-(alpha/2))
    bootstrap_lower_quantile = np.quantile(bootstraps, alpha/2)
    bootstrap_upper_quantile = np.quantile(bootstraps, 1-(alpha/2))

    normal_confidence_interval = (theta_hat - z_alpha_2 * se, theta_hat + z_alpha_2 * se)
    pivotal_confidence_interval = (2*theta_hat - bootstrap_upper_quantile, 2*theta_hat - bootstrap_lower_quantile)
    percentile_confidence_interval = (bootstrap_lower_quantile, bootstrap_upper_quantile)

    if normal_confidence_interval[0] <= true_skewness <= normal_confidence_interval[1]:
        normal_ci_count += 1

    if pivotal_confidence_interval[0] <= true_skewness <= pivotal_confidence_interval[1]:
        pivotal_ci_count += 1
        
    if percentile_confidence_interval[0] <= true_skewness <= percentile_confidence_interval[1]:
        percentile_ci_count += 1

print("Confidence Intervals Coverage:")

print("Normal", normal_ci_count)
print("Pivotal", pivotal_ci_count)
print("Percentile", percentile_ci_count)


Confidence Intervals Coverage:
Normal 1
Pivotal 2
Percentile 0
